In [144]:
# Import dependencies

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import streamlit as st
from joblib import Parallel, delayed
from pandas.tseries.offsets import DateOffset
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from finta import TA

In [145]:
# Import data
market_data_df = pd.read_csv('data/markets_ohlc.csv', header=[0,1], index_col=0)

In [146]:
# Import trained models
svm_SP500 = joblib.load('models/linear_svm_SP 500.pkl')
svm_NASDAQ100 = joblib.load('models/linear_svm_NASDAQ 100.pkl')
svm_RUSSELL2000 = joblib.load('models/linear_svm_RUSSELL 2000.pkl')  

In [147]:
# Declare Constants

# dcb = Dot Com Bubble
dcb_start = '1997-06-01'
dcb_end = '2002-12-01'

# crsh = 2008 Crash
crsh_start = '2007-06-01'
crsh_end = '2012-12-01'

# cvd = COVID-19
cvd_start = '2020-03-01'
cvd_end = '2022-06-01'

short_window = 4
long_window = 100
initial_capital = 100000.0
share_size = 100
start_date = dcb_start
end_date = dcb_end
stock = 'SP 500'
ohlc_df = market_data_df[stock]

In [148]:
def get_ohlc_data(df=ohlc_df, start=dcb_start, end=dcb_end):
    """
    Takes a single dimension OHLC dataframe and returns a copy of it within the 
    boundaries of start and end.
    """
    return df[start:end].copy()

In [149]:
def get_under_over_signals(data=ohlc_df):
    """
    Create a signal based on the current day's closing price being higher or
    lower than yesterdays.

    Returns a date-indexed single column dataframe of the signal.
    """
    df = data.copy()

    df['Actual Returns'] = df['Close'].pct_change()

    df['Signal'] = 0.0
    df['Signal'] = np.where(
        (df['Actual Returns'] >= 0), 1.0, 0.0
    )

    df = df.drop(
        columns=['Close', 'Open', 'Low', 'High', 'Actual Returns']
    )
    df = df.dropna().sort_index(axis='columns')

    return df

In [150]:
def get_fast_slow_sma(data=ohlc_df, short_window=short_window, long_window=long_window):
    """
    Create a signal based on the current day's closing price being higher or
    lower than yesterdays.

    Returns a date-indexed dataframe with SMA Fast, and SMA Slow columns
    """

    df = data.drop(columns=['Open', 'Low', 'High'])

    # Generate the fast and slow simple moving averages
    df['SMA Fast'] = (
        df['Close'].rolling(window=short_window).mean()
    )
    df['SMA Slow'] = (
        df['Close'].rolling(window=long_window).mean()
    )

    # Sort the index
    df = df.drop(columns='Close').dropna().sort_index(axis='columns')

    return df

In [151]:
def get_dmac_signals(data=ohlc_df, short_window=short_window, long_window=long_window):
    """
    Create a signal based on the current day's closing price being higher or
    lower than yesterdays.

    Returns a date-indexed dataframe with SMA Fast, and SMA Slow columns
    """

    df = data.drop(columns=['Open', 'Low', 'High'])

    # Generate the fast and slow simple moving averages
    df['SMA Fast'] = (
        df['Close'].rolling(window=short_window).mean()
    )
    df['SMA Slow'] = (
        df['Close'].rolling(window=long_window).mean()
    )

    # Generate signals based on SMA crossovers
    df['Signal'] = 0.0
    df['Signal'][short_window:] = np.where(
        df['SMA Fast'][short_window:] > 
        df['SMA Slow'][short_window:], 1.0, 0.0
    )

    # df['Entry/Exit'] = df['Signal'].diff()

    # Sort the index
    df = df.drop(columns='Close').dropna().sort_index(axis='columns')

    return df

In [152]:
def get_svm_signals(start=dcb_start, end=dcb_end, stock=stock):

    if stock == 'SP 500':         model = svm_SP500
    elif stock == 'NASDAQ 100':   model = svm_NASDAQ100
    elif stock == 'RUSSELL 2000': model = svm_RUSSELL2000

    # Get the appropriate feature set
    X_data = get_ohlc_data(start=dcb_start, end=dcb_end)
    X = get_fast_slow_sma(X_data)

    X_sc = StandardScaler().fit_transform(X)

    # Use the feature set to predict the target
    y_pred = model.predict(X_sc)

    # get the boundary datestrings of X's date-index
    X_start = X.iloc[0].name
    X_end = X.iloc[-1].name

    # Get the y_true values corresponding to the predicted set
    y_true = get_under_over_signals(X_data[X_start:X_end]).values
    y_true = np.ravel(y_true)

    df = pd.DataFrame({
        'Signal': y_pred,
        'Close': X_data[X_start:X_end]['Close']
    }, index=X.index)

    return df


In [153]:
def calculate_portfolio(data=pd.DataFrame, initial_capital=initial_capital, share_size=share_size):

    df = data.copy()

    initial_capital = float(initial_capital)

    df['Position'] = share_size * df['Signal']
    df['Entry/Exit Position'] = df['Position'].diff()
    df['Holdings'] = df['Close'] * df['Position']
    df['Cash'] = (
        initial_capital - (df['Close'] * df['Entry/Exit Position']).cumsum()
    )
    df['Portfolio Total'] = df['Cash'] + df['Holdings']
    df['Actual Returns'] = df['Close'].pct_change()
    df['Actual Cumulative Returns'] = (
        1 + df['Actual Returns']
    ).cumprod() - 1
    df['Algorithm Returns'] = df['Actual Returns'] * df['Signal']
    df['Algorithm Cumulative Returns'] = (
        1 + df['Algorithm Returns']
    ).cumprod() - 1

    df = df.dropna().sort_index(axis='columns')

    return df

In [154]:
def get_entries_fig(entries=pd.DataFrame):

    entries_fig = px.scatter(entries)
    entries_fig.update_traces(
        marker=dict(
            symbol='triangle-up',
            color='green',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    return entries_fig

In [155]:
def get_exits_fig(exits=pd.DataFrame):

    exits_fig = px.scatter(exits)
    exits_fig.update_traces(
        marker=dict(
            symbol='triangle-down',
            color='red',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    return exits_fig

In [156]:
def plot_trades(data=pd.DataFrame, stock=stock, title='Trades View'):

    df = data.copy()

    df['Entry/Exit'] = df['Signal'].diff()

    entries = df[df['Entry/Exit'] == 1.0]['Close']
    entries.rename('Buy', inplace=True)

    entries_fig = get_entries_fig(entries)

    exits = df[df['Entry/Exit'] == -1.0]['Close']
    exits.rename('Sell', inplace=True)

    exits_fig = get_exits_fig(exits)

    df = df.drop(columns=['Signal', 'Entry/Exit'])

    price_sma_fig = px.line(df)

    all_figs = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_figs.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title
    )

    return all_figs


In [157]:
def plot_portfolio(data=pd.DataFrame, title='Portfolio Performance'):

    df = data.copy()

    df['Entry/Exit'] = df['Signal'].diff()

    entries = df[df['Entry/Exit'] == 1.0]['Portfolio Total']
    entries.rename('Buy', inplace=True)

    entries_fig = get_entries_fig(entries)

    exits = df[df['Entry/Exit'] == -1.0]['Portfolio Total']
    exits.rename('Sell', inplace=True)

    exits_fig = get_exits_fig(exits)

    price_sma_fig = px.line(df[['Portfolio Total']])

    all_fig = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig

In [158]:
def plot_returns(data=pd.DataFrame, title='Portfolio Returns'):
    """
    Plots algorithmic cumulative returns and buy & hold cumulative returns
    Input data must be a df made by `calculate_portfolio()` 
    """

    df = data.copy()

    returns_fig = px.line(
        df[['Actual Cumulative Returns', 'Algorithm Cumulative Returns']]
    )

    all_fig = go.Figure(
        data=returns_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig

In [159]:
svm_signals = get_svm_signals(start=dcb_start, end=dcb_end, stock=stock)
svm_ptf = calculate_portfolio(svm_signals)

/opt/anaconda3/envs/algo/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning:

X has feature names, but LinearSVC was fitted without feature names



In [160]:
plot_trades(svm_signals)

In [161]:
# svm_ptf = calculate_portfolio(svm_signals)
plot_portfolio(svm_ptf)

In [162]:
plot_returns(svm_ptf)